In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
   
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cred-resolve-intelligence-challenge-next-best/sample_submission.csv
/kaggle/input/cred-resolve-intelligence-challenge-next-best/whatsapp_activity.csv
/kaggle/input/cred-resolve-intelligence-challenge-next-best/call_placed.csv
/kaggle/input/cred-resolve-intelligence-challenge-next-best/AI_sms_callback.csv
/kaggle/input/cred-resolve-intelligence-challenge-next-best/mobile_app_data.csv
/kaggle/input/cred-resolve-intelligence-challenge-next-best/train.csv
/kaggle/input/cred-resolve-intelligence-challenge-next-best/metaData.csv
/kaggle/input/cred-resolve-intelligence-challenge-next-best/test.csv
/kaggle/input/cred-resolve-intelligence-challenge-next-best/teleco_call_back.csv


In [2]:
df_meta_data = pd.read_csv("/kaggle/input/cred-resolve-intelligence-challenge-next-best/metaData.csv")
df_meta_data.head()

,lead_code,total_due,dpd_bucket,state
0,LDSMFG_100000,126958,60-90,Karnataka
1,LDSMFG_100001,151867,30-60,UP
2,LDSMFG_100002,136932,60-90,Delhi
3,LDSMFG_100003,370838,30-60,Tamil Nadu
4,LDSMFG_100004,264178,60-90,Delhi


In [3]:
def find_avg(val):
    for i in range(len(val)):
        ans = 0
        if val[i]=="-":
            ans = (int(val[:i])+int(val[i+1:]))/2
            return ans
        else:
            continue

In [4]:
df_meta_data["avg_dpd"] = df_meta_data["dpd_bucket"].apply(find_avg)

In [5]:
df_meta_data.head()

,lead_code,total_due,dpd_bucket,state,avg_dpd
0,LDSMFG_100000,126958,60-90,Karnataka,75.0
1,LDSMFG_100001,151867,30-60,UP,45.0
2,LDSMFG_100002,136932,60-90,Delhi,75.0
3,LDSMFG_100003,370838,30-60,Tamil Nadu,45.0
4,LDSMFG_100004,264178,60-90,Delhi,75.0


In [6]:
new_df_meta_data = df_meta_data[["lead_code","total_due","avg_dpd"]]
new_df_meta_data.head()

,lead_code,total_due,avg_dpd
0,LDSMFG_100000,126958,75.0
1,LDSMFG_100001,151867,45.0
2,LDSMFG_100002,136932,75.0
3,LDSMFG_100003,370838,45.0
4,LDSMFG_100004,264178,75.0


In [7]:
df_teleco_call_back = pd.read_csv("/kaggle/input/cred-resolve-intelligence-challenge-next-best/teleco_call_back.csv")
df_teleco_call_back.head()

,lead_code,call_type,disposition,duration,start_time,transcript_json
0,LDSMFG_169928,BOT,BUSY,62,2025-01-03,{}
1,LDSMFG_118994,BOT,NO_ANSWER,153,2025-01-18,{}
2,LDSMFG_196829,BOT,NO_ANSWER,220,2025-01-12,{}
3,LDSMFG_138730,BOT,NO_ANSWER,196,2025-01-04,{}
4,LDSMFG_175894,BOT,NO_ANSWER,133,2025-01-10,{}


In [8]:
def sentiment(val):
    if len(val)==0:
        return "null"
    else:
        return val.get("sentiment","null")

In [9]:
import ast
df_teleco_call_back["transcript_json"] = df_teleco_call_back["transcript_json"].apply(lambda x:ast.literal_eval(x) if isinstance(x,str) else x)

In [10]:
df_teleco_call_back["sentiment"] = df_teleco_call_back["transcript_json"].apply(sentiment)

In [11]:
df_teleco_call_back.head()

,lead_code,call_type,disposition,duration,start_time,transcript_json,sentiment
0,LDSMFG_169928,BOT,BUSY,62,2025-01-03,{},null
1,LDSMFG_118994,BOT,NO_ANSWER,153,2025-01-18,{},null
2,LDSMFG_196829,BOT,NO_ANSWER,220,2025-01-12,{},null
3,LDSMFG_138730,BOT,NO_ANSWER,196,2025-01-04,{},null
4,LDSMFG_175894,BOT,NO_ANSWER,133,2025-01-10,{},null


In [12]:
df_teleco_call_back["total_calls_made"] = df_teleco_call_back.groupby("lead_code")["lead_code"].transform("count")
df_teleco_call_back["total_answered_calls"] = (df_teleco_call_back["disposition"]=="ANSWERED").groupby(df_teleco_call_back["lead_code"]).transform("sum")
df_teleco_call_back["total_positive_sentiment"] = (df_teleco_call_back["sentiment"]=="positive").groupby(df_teleco_call_back["lead_code"]).transform("sum")
df_teleco_call_back["mean_duration"] = df_teleco_call_back.groupby("lead_code")["duration"].transform("mean")

In [13]:
df_teleco_call_back.head()

,lead_code,call_type,disposition,duration,start_time,transcript_json,sentiment,total_calls_made,total_answered_calls,total_positive_sentiment,mean_duration
0,LDSMFG_169928,BOT,BUSY,62,2025-01-03,{},null,2,1,1,64.000000
1,LDSMFG_118994,BOT,NO_ANSWER,153,2025-01-18,{},null,2,1,1,170.000000
2,LDSMFG_196829,BOT,NO_ANSWER,220,2025-01-12,{},null,1,0,0,220.000000
3,LDSMFG_138730,BOT,NO_ANSWER,196,2025-01-04,{},null,3,1,1,192.666667
4,LDSMFG_175894,BOT,NO_ANSWER,133,2025-01-10,{},null,2,0,0,139.000000


In [14]:
new_df_teleco = df_teleco_call_back[["lead_code","mean_duration","total_calls_made","total_answered_calls","total_positive_sentiment"]]

In [15]:
new_df_teleco.head()
new_df_teleco.duplicated().sum()

np.int64(22346)

In [16]:
new_df_teleco = new_df_teleco.drop_duplicates()
new_df_teleco["positive_sentiment_ratio"] = new_df_teleco["total_positive_sentiment"]/new_df_teleco["total_calls_made"]
new_df_teleco.head()

,lead_code,mean_duration,total_calls_made,total_answered_calls,total_positive_sentiment,positive_sentiment_ratio
0,LDSMFG_169928,64.000000,2,1,1,0.500000
1,LDSMFG_118994,170.000000,2,1,1,0.500000
2,LDSMFG_196829,220.000000,1,0,0,0.000000
3,LDSMFG_138730,192.666667,3,1,1,0.333333
4,LDSMFG_175894,139.000000,2,0,0,0.000000


In [17]:
df_whatsapp = pd.read_csv("/kaggle/input/cred-resolve-intelligence-challenge-next-best/whatsapp_activity.csv")
df_whatsapp.head()

,lead_code,sent_at,status,response_message,read_at
0,LDSMFG_124034,2025-01-10,DELIVERED,NaN,NaN
1,LDSMFG_143014,2025-01-01,DELIVERED,NaN,NaN
2,LDSMFG_192539,2025-01-20,READ,NaN,2025-01-20 11:00:00
3,LDSMFG_141327,2025-01-08,READ,NaN,2025-01-08 12:00:00
4,LDSMFG_163053,2025-01-25,READ,NaN,2025-01-25 10:00:00


In [18]:
df_whatsapp["count_total_messages"] = df_whatsapp.groupby("lead_code")["lead_code"].transform("count")
df_whatsapp["positively_responsed"] = (df_whatsapp["response_message"]=="Pay link?").groupby(df_whatsapp["lead_code"]).transform("sum")

In [19]:
new_df_whatsapp = df_whatsapp[["lead_code","count_total_messages","positively_responsed"]]
new_df_whatsapp = new_df_whatsapp.drop_duplicates()
new_df_whatsapp["positive_response_ratio"] = new_df_whatsapp["positively_responsed"]/new_df_whatsapp["count_total_messages"]
new_df_whatsapp.head()

,lead_code,count_total_messages,positively_responsed,positive_response_ratio
0,LDSMFG_124034,4,0,0.00
1,LDSMFG_143014,4,0,0.00
2,LDSMFG_192539,4,1,0.25
3,LDSMFG_141327,1,0,0.00
4,LDSMFG_163053,5,0,0.00


In [20]:
df_mobile_app = pd.read_csv("/kaggle/input/cred-resolve-intelligence-challenge-next-best/mobile_app_data.csv")
df_mobile_app.head()

,lead_code,visit_date,latitude,longitude,result,remarks
0,LDSMFG_165067,2025-01-08,21.555155,77.611374,MET_CUSTOMER,Visited location
1,LDSMFG_179028,2025-01-23,19.801361,77.559830,SHIFTED,Visited location
2,LDSMFG_103580,2025-01-24,18.338005,77.592332,MET_CUSTOMER,Visited location
3,LDSMFG_114426,2025-01-07,21.868144,77.775007,DOOR_LOCKED,Visited location
4,LDSMFG_170042,2025-01-13,14.782281,77.530485,DOOR_LOCKED,Visited location


In [21]:
df_mobile_app["total_visits"]=df_mobile_app.groupby("lead_code")["lead_code"].transform("count")
df_mobile_app["total_positive_visits"] = (df_mobile_app["result"]=="MET_CUSTOMER").groupby(df_mobile_app["lead_code"]).transform("sum")
df_mobile_app.head()

,lead_code,visit_date,latitude,longitude,result,remarks,total_visits,total_positive_visits
0,LDSMFG_165067,2025-01-08,21.555155,77.611374,MET_CUSTOMER,Visited location,3,2
1,LDSMFG_179028,2025-01-23,19.801361,77.559830,SHIFTED,Visited location,1,0
2,LDSMFG_103580,2025-01-24,18.338005,77.592332,MET_CUSTOMER,Visited location,1,1
3,LDSMFG_114426,2025-01-07,21.868144,77.775007,DOOR_LOCKED,Visited location,1,0
4,LDSMFG_170042,2025-01-13,14.782281,77.530485,DOOR_LOCKED,Visited location,1,0


In [22]:
new_df_mobile_app = df_mobile_app[["lead_code","total_visits","total_positive_visits"]]
new_df_mobile_app = new_df_mobile_app.drop_duplicates()
new_df_mobile_app["successfull_visit_ratio"] = new_df_mobile_app["total_positive_visits"]/new_df_mobile_app["total_visits"]
new_df_mobile_app.head()

,lead_code,total_visits,total_positive_visits,successfull_visit_ratio
0,LDSMFG_165067,3,2,0.666667
1,LDSMFG_179028,1,0,0.000000
2,LDSMFG_103580,1,1,1.000000
3,LDSMFG_114426,1,0,0.000000
4,LDSMFG_170042,1,0,0.000000


In [23]:
df_call_placed = pd.read_csv("/kaggle/input/cred-resolve-intelligence-challenge-next-best/call_placed.csv")
df_call_placed.head()

,lead_code,call_type,disposition,duration,start_time,agent_id
0,LDSMFG_165406,HUMAN,NO_ANSWER,195,2025-01-11,AG_117
1,LDSMFG_193330,HUMAN,NO_ANSWER,208,2025-01-23,AG_146
2,LDSMFG_130724,HUMAN,NO_ANSWER,100,2025-01-29,AG_120
3,LDSMFG_138711,HUMAN,BUSY,89,2025-01-26,AG_101
4,LDSMFG_153922,HUMAN,ANSWERED,229,2025-01-15,AG_133


In [24]:
df_call_placed["total_count_call_placed"] = df_call_placed.groupby("lead_code")["disposition"].transform("count")
df_call_placed["answered_calls_count"] = (df_call_placed["disposition"]=="ANSWERED").groupby(df_call_placed["lead_code"]).transform("sum")
df_call_placed["average_duration"] = df_call_placed.groupby("lead_code")["duration"].transform("mean")
df_call_placed.head()

,lead_code,call_type,disposition,duration,start_time,agent_id,total_count_call_placed,answered_calls_count,average_duration
0,LDSMFG_165406,HUMAN,NO_ANSWER,195,2025-01-11,AG_117,3,1,168.000000
1,LDSMFG_193330,HUMAN,NO_ANSWER,208,2025-01-23,AG_146,2,0,235.000000
2,LDSMFG_130724,HUMAN,NO_ANSWER,100,2025-01-29,AG_120,3,0,94.333333
3,LDSMFG_138711,HUMAN,BUSY,89,2025-01-26,AG_101,1,0,89.000000
4,LDSMFG_153922,HUMAN,ANSWERED,229,2025-01-15,AG_133,2,2,123.000000


In [25]:
new_df_call_placed = df_call_placed[["lead_code","total_count_call_placed","answered_calls_count","average_duration"]]
new_df_call_placed["answered_ratio"] = new_df_call_placed["answered_calls_count"]/new_df_call_placed["total_count_call_placed"]
new_df_call_placed.drop_duplicates()
new_df_call_placed.head()

/tmp/ipykernel_17/2158781784.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_call_placed["answered_ratio"] = new_df_call_placed["answered_calls_count"]/new_df_call_placed["total_count_call_placed"]


,lead_code,total_count_call_placed,answered_calls_count,average_duration,answered_ratio
0,LDSMFG_165406,3,1,168.000000,0.333333
1,LDSMFG_193330,2,0,235.000000,0.000000
2,LDSMFG_130724,3,0,94.333333,0.000000
3,LDSMFG_138711,1,0,89.000000,0.000000
4,LDSMFG_153922,2,2,123.000000,1.000000


In [26]:
df_train = pd.read_csv("/kaggle/input/cred-resolve-intelligence-challenge-next-best/train.csv")
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                80000 non-null  int64  
 1   lead_code         80000 non-null  object 
 2   suggested_action  80000 non-null  object 
 3   TARGET            80000 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.4+ MB


In [27]:
df_train["suggested_action"].unique()

array(['ACTION_HUMAN_CALL', 'ACTION_FIELD', 'ACTION_DIGITAL',
       'ACTION_BOT'], dtype=object)

In [28]:
df_merged = (new_df_meta_data.merge(new_df_teleco,on="lead_code",how="left").
             merge(new_df_mobile_app,on="lead_code",how = "left").
             merge(new_df_whatsapp,on="lead_code",how="left").
            merge(new_df_call_placed,on="lead_code",how = "left"))

In [29]:
df_merged.head()

,lead_code,total_due,avg_dpd,mean_duration,total_calls_made,total_answered_calls,total_positive_sentiment,positive_sentiment_ratio,total_visits,total_positive_visits,successfull_visit_ratio,count_total_messages,positively_responsed,positive_response_ratio,total_count_call_placed,answered_calls_count,average_duration,answered_ratio
0,LDSMFG_100000,126958,75.0,14.5,2.0,0.0,0.0,0.0,NaN,NaN,NaN,3.0,0.0,0.0,NaN,NaN,NaN,NaN
1,LDSMFG_100001,151867,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,140.0,0.0
2,LDSMFG_100001,151867,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,140.0,0.0
3,LDSMFG_100002,136932,75.0,214.0,2.0,0.0,0.0,0.0,NaN,NaN,NaN,3.0,0.0,0.0,2.0,0.0,90.0,0.0
4,LDSMFG_100002,136932,75.0,214.0,2.0,0.0,0.0,0.0,NaN,NaN,NaN,3.0,0.0,0.0,2.0,0.0,90.0,0.0


In [30]:
df_merged = df_merged.drop_duplicates()

In [31]:
df_merged.shape

(100000, 18)

In [32]:
df_merged = df_merged.fillna(0)

In [33]:
df_train = pd.read_csv("/kaggle/input/cred-resolve-intelligence-challenge-next-best/train.csv")
df_test = pd.read_csv("/kaggle/input/cred-resolve-intelligence-challenge-next-best/test.csv")

In [34]:
new_df_train = pd.merge(df_train,df_merged,on="lead_code",how="inner")
new_df_test = pd.merge(df_test,df_merged,on="lead_code",how="inner")

In [35]:
new_df_train.head()

,id,lead_code,suggested_action,TARGET,total_due,avg_dpd,mean_duration,total_calls_made,total_answered_calls,total_positive_sentiment,...,total_visits,total_positive_visits,successfull_visit_ratio,count_total_messages,positively_responsed,positive_response_ratio,total_count_call_placed,answered_calls_count,average_duration,answered_ratio
0,1,LDSMFG_100000,ACTION_HUMAN_CALL,0.47,126958,75.0,14.5,2.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,LDSMFG_100001,ACTION_FIELD,0.35,151867,45.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,140.0,0.0
2,3,LDSMFG_100002,ACTION_HUMAN_CALL,0.09,136932,75.0,214.0,2.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,90.0,0.0
3,4,LDSMFG_100003,ACTION_DIGITAL,0.40,370838,45.0,51.0,1.0,1.0,1.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,LDSMFG_100004,ACTION_HUMAN_CALL,0.49,264178,75.0,114.0,3.0,2.0,2.0,...,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,77.0,0.0


In [36]:
new_df_train["total_due"] = np.log1p(new_df_train["total_due"])
new_df_train = pd.get_dummies(new_df_train,columns=["suggested_action"],dtype = int)
new_df_test = pd.get_dummies(new_df_test,columns=["suggested_action"],dtype = int)

In [37]:
new_df_train.head()

,id,lead_code,TARGET,total_due,avg_dpd,mean_duration,total_calls_made,total_answered_calls,total_positive_sentiment,positive_sentiment_ratio,...,positively_responsed,positive_response_ratio,total_count_call_placed,answered_calls_count,average_duration,answered_ratio,suggested_action_ACTION_BOT,suggested_action_ACTION_DIGITAL,suggested_action_ACTION_FIELD,suggested_action_ACTION_HUMAN_CALL
0,1,LDSMFG_100000,0.47,11.751619,75.0,14.5,2.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
1,2,LDSMFG_100001,0.35,11.930767,45.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,2.0,0.0,140.0,0.0,0,0,1,0
2,3,LDSMFG_100002,0.09,11.827247,75.0,214.0,2.0,0.0,0.0,0.000000,...,0.0,0.0,2.0,0.0,90.0,0.0,0,0,0,1
3,4,LDSMFG_100003,0.40,12.823523,45.0,51.0,1.0,1.0,1.0,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0
4,5,LDSMFG_100004,0.49,12.484382,75.0,114.0,3.0,2.0,2.0,0.666667,...,0.0,0.0,1.0,0.0,77.0,0.0,0,0,0,1


In [38]:
from sklearn.model_selection import train_test_split
X = new_df_train.drop(["id","lead_code","TARGET"],axis = 1)
y = new_df_train["TARGET"]

In [39]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
model = XGBRegressor(n_estimators = 500,learning_rate = 0.05, max_depth = 6,random_state = 42)

In [40]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.15,random_state = 42)
rf_model.fit(X_train,y_train)

RandomForestRegressor(max_depth=10, n_jobs=-1, random_state=42)

In [41]:
from sklearn.metrics import mean_absolute_error,r2_score
val_preds = rf_model.predict(X_test)
error = r2_score(y_test,val_preds)
error

0.23974978723374885

In [42]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error

base_models = [
    ('xgb', XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=5, random_state=42)),
    ('rf', RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)),
    ('lgbm', LGBMRegressor(n_estimators=500, learning_rate=0.05, random_state=42))
]

meta_model = Ridge()

stack_model = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5 # Uses cross-validation to prevent overfitting
)

stack_model.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sqlalchemy/orm/query.py:195: SyntaxWarning: "is not" with 'tuple' literal. Did you mean "!="?
  if entities is not ():


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007953 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 879
[LightGBM] [Info] Number of data points in the train set: 68000, number of used features: 21
[LightGBM] [Info] Start training from score 0.448782
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006612 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 876
[LightGBM] [Info] Number of data points in the train set: 54400, number of used features: 21
[LightGBM] [Info] Start training from score 0.448277
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005396 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough,

StackingRegressor(cv=5,
                  estimators=[('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None,
                                            feature_weights=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_co...
                                            max_delta_step=None, max_depth=5,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=500, n_jobs=None,
                                            num_parallel_tree=None, ...)),
                              ('rf',
                               RandomForestRegressor(max_depth=10,
                                                     n_estimators=200,
                                                     random_state=42)),
                              ('lgbm',
                               LGBMRegressor(learning_rate=0.05,
                                             n_estimators=500,
                                             random_state=42))],
                  final_estimator=Ridge())

In [43]:
stack_preds = stack_model.predict(X_test)

print(f"Stacking R2 Score: {r2_score(y_test, stack_preds):.4f}")
print(f"Stacking MAE: {mean_absolute_error(y_test, stack_preds):.4f}")

Stacking R2 Score: 0.2401
Stacking MAE: 0.1393
